In [ ]:
from twelvelabs import TwelveLabs
import os
from glob import glob
from twelvelabs.models.task import Task
from twelvelabs import APIStatusError
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

api_key = os.getenv('TL_API_KEY')
if not api_key:
    raise ValueError("TL_API_KEY environment variable not set")

client = TwelveLabs(api_key=api_key)

In [ ]:
models = [
        {
          "name": "marengo2.7",
          "options": ["visual", "audio"]
        },
        {
            "name": "pegasus1.2",
            "options": ["visual", "audio"]
        }
    ]


In [ ]:
index = client.index.create(
  name="my-index",
  models=models,
  addons=["thumbnail"] # Optional
)
print(f"A new index has been created: id={index.id} name={index.name} models={index.models}")


In [ ]:
task = client.task.create(
  index_id="67d938f001d86e6384d99851",
  file="videos/video.mp4",
)
print(f"Task id={task.id}, Video id={task.video_id}")


In [ ]:
# Utility function to print the status of a video indexing task
def on_task_update(task: Task):
    print(f"  Status={task.status}")

task.wait_for_done(sleep_interval=5, callback=on_task_update)

if task.status != "ready":
    raise RuntimeError(f"Indexing failed with status {task.status}")
print(f"The unique identifer of your video is {task.video_id}.")


In [ ]:
!pip install ffmpeg

In [ ]:
import os
import subprocess

# Get all videos in an Index
videos = client.index.video.list(index.id)
for video in videos:
    print(f"Processing video {video.id}")
    
    # Get video file path (you'll need to adjust this based on how you access the actual files)
    video_path = f"videos/video.mp4"  # Adjust this path as needed
    
    if not os.path.exists(video_path):
        print(f"Video file not found for {video.id}")
        continue
    
    # Create output directory for trimmed highlights
    output_dir = f"highlights/{video.id}"
    os.makedirs(output_dir, exist_ok=True)
    
    # Get highlights for the video
    print("Extracting highlights and trimming video:")
    res = client.generate.summarize(video_id=video.id, type="highlight")
    
    # Process each highlight
    for i, highlight in enumerate(res.highlights):
        highlight_text = highlight.highlight
        start_time = highlight.start
        end_time = highlight.end
        
        print(f"  Highlight {i+1}: {highlight_text}")
        print(f"    Timestamp: {start_time} to {end_time}")
        
        # Create a descriptive filename (truncate if too long)
        safe_text = "".join(c for c in highlight_text if c.isalnum() or c in " ").strip()
        safe_text = safe_text[:30].strip()  # Limit length
        output_file = f"{output_dir}/highlight_{i+1}_{safe_text}.mp4"
        
        try:
            # Use FFmpeg to trim the video
            # -ss: start time, -to: end time, -c:v copy -c:a copy: copy streams without re-encoding
            cmd = [
                'ffmpeg',
                '-i', video_path,
                '-ss', str(start_time),
                '-to', str(end_time),
                '-c:v', 'copy',
                '-c:a', 'copy',
                output_file,
                '-y'  # Overwrite output file if it exists
            ]
            
            subprocess.run(cmd, check=True, capture_output=True)
            print(f"    Saved highlight to: {output_file}")
        except subprocess.CalledProcessError as e:
            print(f"    Error trimming highlight: {e.stderr.decode()}")
        except Exception as e:
            print(f"    Error: {str(e)}")
    
    # Generate additional metadata for the video
    print(f"Summary: {client.generate.summarize(video_id=video.id, type='summary').summary}")
    
    print("Chapters:")
    res = client.generate.summarize(video_id=video.id, type="chapter")
    for chapter in res.chapters:
        print(
            f"  chapter_number={chapter.chapter_number} chapter_title={chapter.chapter_title} "
            f"start={chapter.start} end={chapter.end}"
        )
    
    res = client.generate.text(video_id=video.id, 
        prompt="Based on this video, I want to generate five keywords for SEO (Search Engine Optimization).")
    print(f"SEO Keywords: {res.data}")